In [2]:
# importing the Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [47]:
i = np.identity(8)

len(i)

['00000001', '00000010', '00000100', '00001000', '00010000', '00100000', '01000000', '10000000']


8

In [50]:
# feed as array input

train_set = np.array(i[:6], dtype='int')
test_set = np.array(i[6:], dtype='int')

train_set = torch.FloatTensor(train_set)

train_set
test_set

tensor([[0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [120]:
# encode into another vector, lower dimension
# uses sigmoid activation function
# w - weight, input and b (bias)

class AutoEncoder(nn.Module):
#     what does self, ?
    def __init__(self, ):
        super(AutoEncoder, self).__init__()
#         what does 20 do, change to 10 and 5?
        self.f1 = nn.Linear(len(i),20)
        self.f2 = nn.Linear(20, 10)
        self.f3 = nn.Linear(10, 20)
        self.f4 = nn.Linear(20,len(i))
        self.activate = nn.Sigmoid()
    
    def forward(self, a):
        a = self.activate(self.f1(a))
        a = self.activate(self.f2(a))
        a = self.activate(self.f3(a))
        a = self.f4(a)
        return a
    
autoenc = AutoEncoder()
criteria = nn.MSELoss()
optimizer = optim.RMSprop(autoenc.parameters(), lr = 0.01, weight_decay = 0.5)
    
    

In [121]:
# training

def predict(set_to_get, training=False):
    t_loss = 0
    s = 0
    for a in range(len(set_to_get)):
        input = Variable(train_set[a]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0)>0:
            output = autoenc(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criteria(output, target)
            mean_corrector = i/float(torch.sum(target.data>0)+1e-10)
            loss.backward()
            t_loss += np.sqrt(float(loss.data)*mean_corrector)
            s += 1
            if training: optimizer.step()
    return t_loss, s

nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    loss = predict(train_set,True)
            
    print('epoch: ', str(epoch), ' loss: ', str((loss[0]/loss[1])[0][0]))

            

epoch:  1  loss:  0.3617823581691125
epoch:  2  loss:  0.4295094789694424
epoch:  3  loss:  0.1909656646653571
epoch:  4  loss:  0.12673505720530046
epoch:  5  loss:  0.027959966215435558
epoch:  6  loss:  0.04114147774391674
epoch:  7  loss:  0.012987368628603396
epoch:  8  loss:  0.005988045100069321
epoch:  9  loss:  0.005864319476732282
epoch:  10  loss:  0.0029145950525111017


In [122]:
test_loss = predict(test_set)
print("test_loss = ", str((test_loss[0]/test_loss[1])[0][0]))

test_loss =  0.003360726490098086
